In [77]:
import os
import numpy as np
import json

import nltk
nltk.download('punkt')

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')  + ['shh', 'hoo', 'boo', 'uhoh', 'aah', 'heh', 'huh', 'ooh', 'yo']

[nltk_data] Downloading package punkt to /home/angelo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [78]:
path = "/home/angelo/Desktop/master_offline/progettone_github/Animation-studios-analysis/sottotitoli/clean_subtitles/Disney/Animation/"

all_text = []
for r, d, files in os.walk(path):
    for f in files:
        filename = r+'/'+f
        with open(filename, 'r', encoding ='utf-8', errors='ignore') as fh:
            text = fh.read()
            all_text.append(text)

In [79]:
len(all_text)

88

# sentence splitting

In [122]:
one_document = ' '.join(all_text).lower()
one_document = one_document.replace('\n', ' ')
sentences = sent_tokenize(one_document)

In [123]:
len(sentences)

111766

In [124]:
sentences[0]

'long ago, in the faraway land of ancient greece, there was a golden age of powerful gods and extraordinary heroes.'

In [162]:
tok_sents = [[word for word in word_tokenize(sentence) if (word.isalpha() and word not in en_stopwords)] for sentence in sentences]

In [163]:
tok_sents[0]

['long',
 'ago',
 'faraway',
 'land',
 'ancient',
 'greece',
 'golden',
 'age',
 'powerful',
 'gods',
 'extraordinary',
 'heroes']

In [127]:
tok_sents[0:3]

[['long',
  'ago',
  'faraway',
  'land',
  'ancient',
  'greece',
  'golden',
  'age',
  'powerful',
  'gods',
  'extraordinary',
  'heroes'],
 ['greatest', 'strongest', 'heroes', 'mighty', 'hercules'],
 ['measure', 'true', 'hero']]

In [ ]:
from collections import Counter
counter = Counter([len(s) for s in sentences])
sorted(counter.items())

In [129]:
[s for s in sentences if len(s) == 908]

["you're jokin'  you're jokin'  i can't believe  my eyes  you're jokin' me  you gotta be  this can't be  the right guy  he's ancient  he's ugly  i don't know  which is worse  i might just split a seam now  if i don't die laughin' first  mr. oogie boogie says  there's trouble close at hand  you better  pay attention now  'cause i'm  the boogie man  and if you  aren't shakin'  there's somethin'  very wrong  'cause this may be the last time  you hear the boogie song  whoaoh  wahah  whoaho  ow  whoaoh  wow  i'm the oogie boogie man  release me now or you must  face the dire consequences  the children are expecting me  so please, come to your senses  ah, you're jokin'  you're jokin'  i can't believe  my ears  would someone  shut this fella up  i'm drownin'  in my tears  he's funny  i'm laughin'  you really are  too much  and now  with your permission  i'm goin' to do my stuff  what are you going to do?"]

# WORD2VEC

In [130]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [149]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(tok_sents, size=200, window=10, min_count=15, sg=0, iter=20, negative=10)

2020-06-13 01:05:01,135 : INFO : collecting all words and their counts
2020-06-13 01:05:01,137 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-13 01:05:01,154 : INFO : PROGRESS: at sentence #10000, processed 54385 words, keeping 5208 word types
2020-06-13 01:05:01,171 : INFO : PROGRESS: at sentence #20000, processed 110999 words, keeping 8498 word types
2020-06-13 01:05:01,180 : INFO : PROGRESS: at sentence #30000, processed 164371 words, keeping 10730 word types
2020-06-13 01:05:01,189 : INFO : PROGRESS: at sentence #40000, processed 218007 words, keeping 12266 word types
2020-06-13 01:05:01,201 : INFO : PROGRESS: at sentence #50000, processed 273017 words, keeping 14271 word types
2020-06-13 01:05:01,211 : INFO : PROGRESS: at sentence #60000, processed 325122 words, keeping 15843 word types
2020-06-13 01:05:01,221 : INFO : PROGRESS: at sentence #70000, processed 375894 words, keeping 17106 word types
2020-06-13 01:05:01,232 : INFO : PROGRESS: at sen

In [150]:
#learned vocabulary
words = list(w2v_model.wv.vocab)
len(words)

2725

In [151]:
w2v_model.wv.most_similar(['aurora'])

2020-06-13 01:05:14,025 : INFO : precomputing L2-norms of word weight vectors


[('hail', 0.8739173412322998),
 ('princess', 0.7359708547592163),
 ('queen', 0.7139188647270203),
 ('health', 0.6875106692314148),
 ('atta', 0.6720589995384216),
 ('gift', 0.6645910143852234),
 ('beloved', 0.6577285528182983),
 ('arendelle', 0.6089876294136047),
 ('yudhishtir', 0.5904470682144165),
 ('king', 0.583905041217804)]

In [152]:
w2v_model.wv.most_similar(['king', 'princess'], ['prince'])

[('aurora', 0.6322866678237915),
 ('hail', 0.629346489906311),
 ('queen', 0.6060227751731873),
 ('gods', 0.5343604683876038),
 ('england', 0.4813360869884491),
 ('gift', 0.47157424688339233),
 ('health', 0.45906397700309753),
 ('arendelle', 0.44755175709724426),
 ('duryodhan', 0.4470111131668091),
 ('arrived', 0.4429568648338318)]

In [153]:
w2v_model.wv.similarity('king', 'majesty')

0.37860382

# constructin matrix for graph

In [154]:
voc = set(w2v_model.wv.vocab)
len(voc)

2725

In [155]:
voc = voc.difference(en_stopwords)
len(voc)

2591

In [156]:
indici = dict()
for index, key in enumerate(voc):
    if key not in indici:
        indici[key] = index
        
reverse_indici = dict()
for term, termID in indici.items():
    reverse_indici[termID] = term

In [157]:
#costruzione matrice
graph_matrix = np.zeros(shape=(len(voc), len(voc)), dtype=np.float32)

for index1, key1 in enumerate(voc):
    for index2, key2 in enumerate(voc):
        graph_matrix[index1 ][index2] = w2v_model.wv.similarity(key1, key2)
        
len(graph_matrix)

2591

In [160]:
#constructing the list of tuples
graph_list = []
nodes = set() # <-- NUOVO
for x in range(len(graph_matrix)):
    for y in range(len(graph_matrix)):
        if graph_matrix[x][y] >= 0.7 and x != y:
            nodes |= set([reverse_indici[x], reverse_indici[y]])
            graph_list.append((reverse_indici[x],reverse_indici[y],graph_matrix[x][y]))      
print('nodes:', len(nodes))
print('links:', len(graph_list))

nodes: 276
links: 506


In [161]:
# NUOVO
graph = dict()
graph["nodes"] = [{"id": t} for t in nodes]
graph["links"] = [{"source":t[0], "target":t[1], "value":float(t[2])} for t in graph_list]
with open('data.json', 'w') as outfile:
    json.dump(graph, outfile)

## FastText

In [ ]:
from gensim.models import FastText
fast_model = FastText(tok_sents,size=100, window=10, min_count=5, sg=1, iter=20, negative=10)

In [ ]:
fast_model.wv.most_similar(['apple'])

In [ ]:
fast_model.wv.most_similar(['king'])

In [ ]:
fast_model.wv.most_similar(['mirror'])

In [ ]:
fasttext_wAp_analogy = fast_model.wv.evaluate_word_analogies(test_file, dummy4unknown=True)

# google precomputed word embeddings

In [ ]:
import gensim
google_w2v_file = 'GoogleNews-vectors-negative300.bin'
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_w2v_file,binary=True)

In [ ]:
google_model.most_similar(['king','female'],['male'])

In [ ]:
w2v_large_analogy = google_model.wv.accuracy(test_file)